<a href="https://colab.research.google.com/github/DYLPICKLEROHAN/previous_work/blob/main/2023_big_data_analysis_and_industry_project/mutual_friend_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=2cb5075416ce7a5e5556a7f474732a394a062a428aadb092c293ae7e85463e56
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


Below demonstrated are a few operations that can a head-start to understand how map reduce operations can be performed on an example graph of 6 nodes. More refinement and improvements can be made using this.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from itertools import combinations

# Initialize Spark context
sc = SparkSession.builder.appName('recommendation_system').getOrCreate()
df = sc.read.csv('/content/drive/MyDrive/aabigdatafiles/soc-LiveJournal1Adj.txt', sep = '\t', inferSchema=True, )
df= df.withColumn('stops', F.from_json('stops', ArrayType(StringType())))

ConnectionRefusedError: ignored

In [ ]:
# create an RDD from the adjacency list
rdd = df.rdd.map(lambda x: (x[0], x[1]))
rdd = rdd.filter(lambda x: x[1] is not None)
print("Confirming output format", rdd.filter(lambda x: x[0] == 924).collect())

Confirming output format [(924, '925,926')]


In [ ]:
# define the map function
def map_function(x):
    fromUser = x[0]
    toUser_list = x[1]
    result = []
    for toUser in toUser_list:
        if toUser == fromUser:
            for friend in toUser_list:
                result.append((fromUser, friend, -1))
        else:
          for x in toUser_list:
            if x == toUser:
              continue
            else:
              #result.append( (toUser, x, fromUser) )
              result.append( (toUser, x, 1) )

    return result

# apply the map function to the adjacency list
mapped_rdd = rdd.flatMap(map_function)

# print the output
print(mapped_rdd.collect())

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60382)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-af26b1fc24b4>", line 24, in <module>
    print(mapped_rdd.collect())
  File "/usr/local/lib/python3.9/dist-packages/pyspark/rdd.py", line 1197, in collect
    sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.9/dist-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.9/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <u

ConnectionRefusedError: ignored

In [ ]:
# reduce by key to calculate the total number of mutual friends between each pair of users
reduced_rdd = mapped_rdd.map(lambda x: ((x[0], x[1]), x[2]))
print( reduced_rdd.collect() )

reduced_rdd1 = reduced_rdd.reduceByKey(lambda x, y: x+y )
print( reduced_rdd1.collect() )

reduced_rdd2 = reduced_rdd1.map(lambda x: (x[0][0], x[0][1], x[1]))
print( reduced_rdd2.collect() )

# sort the result based on the number of mutual friends
sorted_rdd = reduced_rdd2.sortBy(lambda x: -x[2])

# print the output
print(sorted_rdd.collect())

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: ignored

In [ ]:
result = sorted_rdd.map(lambda x: (x[0], (x[1], x[2]))).groupByKey()

for (node, rec_friends) in result.collect():
    formatted_friends = [f"{f[0]}({f[1]})" for f in rec_friends]
    print(f"{node} - [{', '.join(formatted_friends)}]")

NameError: ignored

In [ ]:
# Stopping Spark-Session and Spark context
sc.stop()